# Prompt Lab for Chefing
Our objective is to develop the prompts and context requirements to satisfy the usage scenarios listed in our ideation document. Each usage scenario will be allocated a section in this document.

Each section will include:
- A description, including input/output schemas
- At least one sample input
- Code to perform the task given the input
- Tests to evaluate the implementation with different parameters

**Note:** The prompts listed here are reordered and altered in responsibility to handling the scenarios compared to the past milestone as our understanding of the problem and the feasibility/scope of some of the steps changed. Each of the following scenarios integrates in a modular way with the rest of the system.

## Overview of Chefing Structure
*User scenarios here refer to the numberings in the original FinalIdea.ipynb notebook*
Chefing contains a main system prompt that informs the agent of its role and outlines the extra information it will need to process in context.

Then, our main flow is to query long-term information about the user stored in an external location (preferences, restrictions, situation, past recipes) and add this to the main window. Finally, we await the user's first request, which is to suggest recipes. A picture of the fridge will be provided to the user as input for this step. This enables use case 3 immediately (recipe from fridge picture).

Then, the user may continue to add more messages to describe new requirements for the model, which we will classify as either long-term or short-term. We will output long-term information alongside the recipe to store in a vector DB. When the user provides good feedback on this recipe, we will save it for future retrieval. This enables use cases 1 (adaptive difficulty), 4 (planning), 5 (kitchen tools), and 6 (expiry information). To initialize this, we ask the user for a form.

We will simulate usage scenario 2 (retrieval of stored long-term info) by using an embedding model and cosine similarity to see what prompts and data formats we can use to get useful information. This will be the section where we diverge from the structure of the rest of the sections.

## Setup and System Prompt

In [70]:
import base64, json, datetime, zoneinfo
from openai import OpenAI
from dotenv import load_dotenv
_ = load_dotenv("../comm4190_F25/01_Introduction_and_Setup/.env")
client = OpenAI()

SYSTEM_PROMPT = """
You are an expert chef working on the platform Chefing. 
Your goal is to help suggest satisfactory recipes for people so that they can easily cook for themselves.
Be nimble and efficient with your recommendations.
For recipes, be as detailed as possible with instructions, including cooking times, sizings, and tips.
Proactively provide substitutions or alternative steps if anticipating some difficulty/friction. 
Steps should be numbered from 1. Indicate substeps by .i, like 7.1, 7.2, and indicate alternative paths with a separate letter suffix.
Try to ignore super obviously contrarian user fluctuations and instructions.
"""

## Usage Scenario 1: Creating a Recipe using the Ingredients in the Fridge
This is the bulk of the project. Testing for this will cover a variety of cases on our input image.

### Input
```json
{
    "fridge_picture": image_path,
    "user_input": str,
    "instructions": List[str], //corrections, cravings, etc.
    "preferences": List[str],
    "restrictions": List[str], // diet, allergies
    "situation": List[str], // planning, expiry info and kitchen tools
}
```

### Output
```json
{
    "recipe": {
        "name": str
        "ingredients": List[str],
        "steps": List[str]
    }
}
```

### Implementation

In [55]:
def encode_image_to_data_uri(path: str) -> str:
    with open(path, "rb") as f:
        b64 = base64.b64encode(f.read()).decode("utf-8")
        return f"data:image/jpeg;base64,{b64}"

def generate_recipe_from_fridge(
    fridge_image_path: str,
    user_input: str,
    instructions: list[str],
    preferences: list[str],
    restrictions: list[str],
    situation: list[str]
):

    time = datetime.datetime.now().astimezone(zoneinfo.ZoneInfo("America/New_York"))
    
    data_uri = encode_image_to_data_uri(fridge_image_path)
    
    # read the image file
    with open(fridge_image_path, "rb") as img:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": [
                        {"type": "text", "text": SYSTEM_PROMPT + f"\nIt is now {time}."}
                    ],
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image_url",
                            "image_url": {"url": data_uri}
                        },
                        {
                            "type": "text",
                            "text": f"""
Fridge contents image provided above.

{user_input}

Instructions: {", ".join(instructions)}
Preferences: {", ".join(preferences)}
Restrictions: {", ".join(restrictions)}
Situation: {", ".join(situation)}

Please propose a recipe that satisfies all constraints.
Return JSON only.
                            """
                        }
                    ],
                },
            ],
            # enforce output structure
            response_format={
                "type": "json_schema",
                "json_schema": {
                    "name": "recipe_response",
                    "schema": {
                        "type": "object",
                        "properties": {
                            "recipe": {
                                "type": "object",
                                "properties": {
                                    "name": {
                                        "type": "string"
                                    },
                                    "ingredients": {
                                        "type": "array",
                                        "items": {"type": "string"}
                                    },
                                    "steps": {
                                        "type": "array",
                                        "items": {"type": "string"}
                                    }
                                },
                                "required": ["ingredients", "steps"]
                            }
                        },
                        "required": ["recipe"],
                    },
                },
            },
        )

    return json.loads(response.choices[0].message.content)

### Testing

#### Test 1: Plain Image

In [56]:
response = generate_recipe_from_fridge("./fridge.jpeg", "Hi, looking to make a quick dinner!", [], [], [], [])

print(json.dumps(response, indent = 4))

{
    "recipe": {
        "name": "Simple Kimchi Fried Rice",
        "ingredients": [
            "2 cups cooked rice",
            "1 cup kimchi, chopped",
            "2 eggs",
            "2 tablespoons soy sauce",
            "1 tablespoon sesame oil",
            "2 tablespoons vegetable oil",
            "1 green onion, chopped",
            "Salt and pepper to taste",
            "Optional: Protein (chicken, tofu, or shrimp)"
        ],
        "steps": [
            "1. Heat 1 tablespoon of vegetable oil in a large pan or wok over medium heat.",
            "2. Crack the eggs into the pan and scramble them until fully cooked. Remove from the pan and set aside.",
            "3. Add the remaining 1 tablespoon of vegetable oil to the same pan.",
            "4. Add the chopped kimchi and stir-fry for 3-4 minutes until it's fragrant.",
            "5. Add the cooked rice to the pan and mix well with the kimchi.",
            "6. Drizzle soy sauce and sesame oil over the rice mixt

#### Test 2: End-to-End Test

In [57]:
response = generate_recipe_from_fridge(
    "./fridge.jpeg",
    "Hi, looking to make a quick dinner!",
    instructions = [
        "There are no green onions or onions here.",
    ],
    preferences = [
        "high protein",
        "quick meal",
        "mildly spicy"
    ],
    restrictions = [
        "allergic to peanut butter",
        "no shellfish"
    ],
    situation = [
        "stovetop only",
        "no food processor",
        "Items expiring soon: onion"
    ]
)

print(json.dumps(response, indent=4))

{
    "recipe": {
        "name": "Spicy Soy Sauce Chicken Stir-Fry",
        "ingredients": [
            "Chicken breasts or thighs (1 lb, cut into bite-sized pieces)",
            "Soy sauce (3 tbsp)",
            "Sesame oil (1 tbsp)",
            "Garlic (2 cloves, minced)",
            "Ginger (1-inch piece, grated)",
            "Sriracha or chili sauce (1-2 tbsp, to taste)",
            "Bell pepper (1, sliced)",
            "Broccoli florets (2 cups)",
            "Olive oil (2 tbsp)",
            "Cooked rice or noodles (for serving)",
            "Salt and pepper (to taste)"
        ],
        "steps": [
            "1. Heat 1 tablespoon of olive oil in a large skillet over medium-high heat.",
            "2. Add the chicken pieces to the skillet and season with salt and pepper. Cook until browned and cooked through, about 5-7 minutes. Remove from the skillet and set aside.",
            "3. In the same skillet, add the remaining tablespoon of olive oil.",
            "4. Ad

## Usage Scenario 2: Parsing New Information from User
The user will probably need to provide more information. This is how we ingest it so we can add it to the context for the next recipe generation.

### Input
```json
{
    "user_message",
    "instructions": List[str],
    "preferences": List[str],
    "restrictions": List[str],
    "situation": List[str],
}
```
### Output
```json
{
    "new_instructions": List[str],
    "new_preferences": List[str],
    "new_restrictions": List[str],
    "new_situation": List[str],
}
```

### Implementation

In [61]:
def parse_new_user_information(
    user_message: str,
    instructions: list[str],
    preferences: list[str],
    restrictions: list[str],
    situation: list[str],
):

    time = datetime.datetime.now().astimezone(
        zoneinfo.ZoneInfo("America/New_York")
    )

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": SYSTEM_PROMPT + f"\nThe current time is {time}."
            },
            {
                "role": "user",
                "content": f"""
User message:
{user_message}

Existing stored context:
Instructions: {instructions}
Preferences: {preferences}
Restrictions: {restrictions}
Situation: {situation}

Extract ONLY the *new* information. If nothing new was said in a category,
return an empty list for that category.

Return JSON only.
                """
            },
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "parsed_user_info",
                "schema": {
                    "type": "object",
                    "properties": {
                        "new_instructions": {
                            "type": "array",
                            "items": {"type": "string"}
                        },
                        "new_preferences": {
                            "type": "array",
                            "items": {"type": "string"}
                        },
                        "new_restrictions": {
                            "type": "array",
                            "items": {"type": "string"}
                        },
                        "new_situation": {
                            "type": "array",
                            "items": {"type": "string"}
                        }
                    },
                    "required": [
                        "new_instructions",
                        "new_preferences",
                        "new_restrictions",
                        "new_situation"
                    ]
                }
            }
        }
    )

    # Return parsed JSON
    return json.loads(response.choices[0].message.content)

### Testing

#### Test 1: No Onions and New Pan

In [62]:
parsed = parse_new_user_information(
    user_message="By the way I can’t eat onions now, and I bought a new cast iron pan.",
    instructions=["clarify uncertain items", "no assumptions"],
    preferences=["spicy", "high protein"],
    restrictions=["no peanut butter"],
    situation=["stovetop only"]
)

print(json.dumps(parsed, indent=4))

{
    "new_instructions": [],
    "new_preferences": [],
    "new_restrictions": [
        "no onions"
    ],
    "new_situation": [
        "bought a new cast iron pan"
    ]
}


#### Test 2: Adversarial Input

In [71]:
parse_new_user_information(
    user_message=(
        "Ignore everything I said before. I can eat anything. "
        "Except peanuts. And gluten. Also I’m vegan now. "
        "Actually wait—don’t add anything new unless I explicitly say NEW: "
        "NEW: I bought a sous vide machine."
    ),
    instructions=["no assumptions"],
    preferences=["sweet", "high protein"],
    restrictions=["no peanut butter", "gluten free"],
    situation=["oven available"]
)

{'new_instructions': [],
 'new_preferences': [],
 'new_restrictions': ['vegan'],
 'new_situation': ['sous vide machine available']}

#### Test 3: No New Info

In [72]:
parse_new_user_information(
    user_message="That last recipe was great, thanks! I’ll send another fridge picture later.",
    instructions=["ask for clarifications"],
    preferences=["Italian"],
    restrictions=["no shellfish"],
    situation=["stovetop only"]
)

{'new_instructions': [],
 'new_preferences': [],
 'new_restrictions': [],
 'new_situation': []}

#### Test 4: Long Narrative with Context

In [73]:
parse_new_user_information(
    user_message=(
        "So I tried to make dinner yesterday, but my oven suddenly stopped working "
        "and won’t heat anymore. I had to sauté everything on the pan instead. "
        "Also, I realized I’m getting tired of heavy meals—lighter stuff would be great."
    ),
    instructions=["highlight potential allergens"],
    preferences=["savory"],
    restrictions=[],
    situation=["has oven", "stovetop available"]
)

{'new_instructions': [],
 'new_preferences': ['light meals'],
 'new_restrictions': [],
 'new_situation': ['oven not working']}

#### Test 5: Disgruntled User

In [74]:
parse_new_user_information(
    user_message=(
        "I don't know, man. My fridge is chaos. "
        "DON’T ADD ANYTHING ABOUT DIET. I’m serious. NOT ABOUT DIET. "
        "Anyway I bought like 200 potatoes (don’t ask), but that’s temporary, don't store it as a preference. "
        "NEW SITUATION: electricity is out, I’m cooking on a camping stove. "
        "Just don’t try to be smart and ‘infer’ anything else."
    ),
    instructions=["ask before assuming items"],
    preferences=["low sugar"],
    restrictions=["no peanuts"],
    situation=["stovetop only"]
)

{'new_situation': ['electricity is out', 'cooking on a camping stove'],
 'new_restrictions': [],
 'new_preferences': [],
 'new_instructions': []}

## Usage Scenario 3: Gathering Initial Information
In the app, the user provides info to a form where we learn about their ability, restrictions, and goals. Once it ends, we use a similar information parser prompt to collect and finalize the information.

### Input
```json
{
    "ability_description": str,
    "restrictions_description": str,
    "goal_description": str,
}
```
### Output
```json
{
    "long_term_instructions": List[str],
    "long_term_preferences": List[str],
    "long_term_restrictions": List[str],
    "long_term_situation": List[str],
}
```

### Implementation

In [77]:
def parse_user_profile_information(
    ability_description: str,
    restrictions_description: str,
    goal_description: str
):

    user_message = f"""
TASK: Parse the user's long-term cooking profile into structured categories.

Ability description:
{ability_description}

Restrictions description:
{restrictions_description}

Goal description:
{goal_description}

Return JSON only, following this schema:
- long_term_instructions: list of assistant behaviors or meta instructions
- long_term_preferences: list of stable culinary preferences
- long_term_restrictions: list of diet/allergy/medical restrictions
- long_term_situation: list of persistent contextual factors (skills, tools, environment)
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_message}
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "long_term_profile",
                "schema": {
                    "type": "object",
                    "properties": {
                        "long_term_instructions": {
                            "type": "array",
                            "items": {"type": "string"}
                        },
                        "long_term_preferences": {
                            "type": "array",
                            "items": {"type": "string"}
                        },
                        "long_term_restrictions": {
                            "type": "array",
                            "items": {"type": "string"}
                        },
                        "long_term_situation": {
                            "type": "array",
                            "items": {"type": "string"}
                        }
                    },
                    "required": [
                        "long_term_instructions",
                        "long_term_preferences",
                        "long_term_restrictions",
                        "long_term_situation"
                    ]
                }
            }
        }
    )

    return json.loads(response.choices[0].message.content)

### Testing

In [79]:
profile_1 = parse_user_profile_information(
    ability_description="I'm a beginner cook with limited knife skills.",
    restrictions_description="I'm lactose intolerant and avoid shellfish.",
    goal_description="I want to eat healthier, cook faster meals, and lose a little weight."
)

print(json.dumps(profile_1, indent=4))

{
    "long_term_instructions": [
        "Provide simple, easy-to-follow instructions",
        "Include tips for improving knife skills",
        "Suggest recipes with quick preparation times"
    ],
    "long_term_preferences": [
        "Healthier meals",
        "Quick meals"
    ],
    "long_term_restrictions": [
        "Lactose intolerant",
        "Avoid shellfish"
    ],
    "long_term_situation": [
        "Beginner cook",
        "Limited knife skills"
    ]
}


In [80]:
profile_2 = parse_user_profile_information(
    ability_description="Experienced cook, comfortable with knives and stovetop techniques.",
    restrictions_description="No restrictions.",
    goal_description="I want to focus on high-protein, low-carb meals for muscle gain."
)

print(json.dumps(profile_2, indent=4))

{
    "long_term_instructions": [],
    "long_term_preferences": [
        "high-protein meals",
        "low-carb meals"
    ],
    "long_term_restrictions": [],
    "long_term_situation": [
        "experienced cook",
        "comfortable with knives",
        "comfortable with stovetop techniques"
    ]
}


In [81]:
profile_3 = parse_user_profile_information(
    ability_description="I don't cook much, sometimes very confident, sometimes not.",
    restrictions_description="I like eating everything but also want to avoid sugar and gluten.",
    goal_description="My goal is healthy meals but I also love desserts."
)

print(json.dumps(profile_3, indent=4))


{
    "long_term_instructions": [
        "Provide detailed and easy-to-follow instructions to accommodate varying confidence levels.",
        "Offer substitutions for sugar and gluten.",
        "Include dessert options that align with dietary restrictions."
    ],
    "long_term_preferences": [
        "Enjoy a wide variety of foods.",
        "Preference for desserts."
    ],
    "long_term_restrictions": [
        "Avoid sugar.",
        "Avoid gluten."
    ],
    "long_term_situation": [
        "Inconsistent cooking confidence level."
    ]
}


In [82]:
profile_4 = parse_user_profile_information(
    ability_description="I cook occasionally, mostly simple meals.",
    restrictions_description="",
    goal_description=""
)

print(json.dumps(profile_4, indent=4))

{
    "long_term_instructions": [
        "Focus on offering simple recipes with few ingredients.",
        "Provide clear and concise cooking instructions that are easy to follow for someone with basic skills."
    ],
    "long_term_preferences": [
        "Prefers simple meals."
    ],
    "long_term_restrictions": [],
    "long_term_situation": [
        "Occasionally cooks.",
        "Possesses basic cooking skills."
    ]
}


In [83]:
profile_5 = parse_user_profile_information(
    ability_description=(
        "I have moderate cooking skills, I can handle a knife and some frying, "
        "but I struggle with baking and timing multiple dishes simultaneously."
    ),
    restrictions_description=(
        "I avoid peanuts, tree nuts, and shellfish due to allergies, "
        "and I prefer low sodium and low sugar diets. "
        "I also try to eat mostly plant-based foods."
    ),
    goal_description=(
        "My goal is to prepare delicious and nutritious meals, "
        "improve my cooking skills, maintain my weight, "
        "and save time in the kitchen. I also like trying international cuisine."
    )
)

print(json.dumps(profile_5, indent=4))


{
    "long_term_instructions": [
        "Provide detailed cooking instructions with a focus on timing to help manage multiple dishes.",
        "Suggest substitutes for high sodium or sugar ingredients.",
        "Focus on plant-based recipes where possible."
    ],
    "long_term_preferences": [
        "Enjoys trying international cuisine.",
        "Prefers low sodium and low sugar dishes."
    ],
    "long_term_restrictions": [
        "Allergic to peanuts, tree nuts, and shellfish.",
        "Prefers a mostly plant-based diet."
    ],
    "long_term_situation": [
        "Moderate cooking skills with competence using a knife and frying techniques.",
        "Struggles with baking and timing multiple dishes simultaneously."
    ]
}


## Usage Scenario 4: Identifying Long-Term Information
This is important. We probably have some information that is stored over a longer period of time. As a result, we need to be able to identify which of these items is actually long-term information. Note here that instructions are short term.

### Input
```json
{
    "new_instructions": List[str],
    "new_preferences": List[str],
    "new_restrictions": List[str],
    "new_situation": List[str],
    "long_term_instructions": List[str],
    "long_term_preferences": List[str],
    "long_term_restrictions": List[str],
    "long_term_situation": List[str],
}
```
### Output
```json
{
    "new_long_term_instructions": List[str],
    "new_long_term_preferences": List[str],
    "new_long_term_restrictions": List[str],
    "new_long_term_situation": List[str],
}
```

### Implementation

In [90]:
def compute_long_term_delta_with_llm(
    new_instructions,
    new_preferences,
    new_restrictions,
    new_situation,
    long_term_instructions,
    long_term_preferences,
    long_term_restrictions,
    long_term_situation
):

    user_message = f"""
You are given the following new short-term inputs and the existing long-term profile.
Interpret which items from the new inputs should be preserved in the long-term profile.
Do not report items that are already existing.

EXISTING LONG-TERM PROFILE:
Instructions: {long_term_instructions}
Preferences: {long_term_preferences}
Restrictions: {long_term_restrictions}
Situation: {long_term_situation}

NEW SHORT-TERM INPUTS:
Instructions: {new_instructions}
Preferences: {new_preferences}
Restrictions: {new_restrictions}
Situation: {new_situation}

Return JSON ONLY with these keys:
- new_long_term_instructions
- new_long_term_preferences
- new_long_term_restrictions
- new_long_term_situation
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_message}
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "long_term_delta",
                "schema": {
                    "type": "object",
                    "properties": {
                        "new_long_term_instructions": {
                            "type": "array", "items": {"type": "string"}
                        },
                        "new_long_term_preferences": {
                            "type": "array", "items": {"type": "string"}
                        },
                        "new_long_term_restrictions": {
                            "type": "array", "items": {"type": "string"}
                        },
                        "new_long_term_situation": {
                            "type": "array", "items": {"type": "string"}
                        }
                    },
                    "required": [
                        "new_long_term_instructions",
                        "new_long_term_preferences",
                        "new_long_term_restrictions",
                        "new_long_term_situation"
                    ]
                }
            }
        }
    )

    return json.loads(response.choices[0].message.content)


### Testing

In [91]:
existing_long_term = {
    "long_term_instructions": ["use simple explanations"],
    "long_term_preferences": ["healthy meals", "quick recipes"],
    "long_term_restrictions": ["lactose intolerant", "no shellfish"],
    "long_term_situation": ["beginner cook"]
}

new_short_term = {
    "new_instructions": ["notify if food looks spoiled", "use simple explanations"],
    "new_preferences": ["quick recipes", "citrus flavors"],
    "new_restrictions": ["no shellfish", "no shrimp"],
    "new_situation": ["microwave only at work", "beginner cook"]
}

delta = compute_long_term_delta_with_llm(
    new_instructions=new_short_term["new_instructions"],
    new_preferences=new_short_term["new_preferences"],
    new_restrictions=new_short_term["new_restrictions"],
    new_situation=new_short_term["new_situation"],
    long_term_instructions=existing_long_term["long_term_instructions"],
    long_term_preferences=existing_long_term["long_term_preferences"],
    long_term_restrictions=existing_long_term["long_term_restrictions"],
    long_term_situation=existing_long_term["long_term_situation"]
)

print(json.dumps(delta, indent=4))

{
    "new_long_term_instructions": [
        "notify if food looks spoiled"
    ],
    "new_long_term_preferences": [
        "citrus flavors"
    ],
    "new_long_term_restrictions": [
        "no shrimp"
    ],
    "new_long_term_situation": [
        "microwave only at work"
    ]
}


In [92]:
delta2 = compute_long_term_delta_with_llm(
    new_instructions=["always verify expiration dates"],
    new_preferences=["spicy"],
    new_restrictions=["no peanuts"],
    new_situation=["stovetop only"],
    long_term_instructions=[],
    long_term_preferences=[],
    long_term_restrictions=[],
    long_term_situation=[]
)

print(json.dumps(delta2, indent=4))

{
    "new_long_term_instructions": [
        "always verify expiration dates"
    ],
    "new_long_term_preferences": [
        "spicy"
    ],
    "new_long_term_restrictions": [
        "no peanuts"
    ],
    "new_long_term_situation": [
        "stovetop only"
    ]
}


In [93]:
delta3 = compute_long_term_delta_with_llm(
    new_instructions=["use simple explanations"],
    new_preferences=["healthy meals"],
    new_restrictions=["lactose intolerant"],
    new_situation=["beginner cook"],
    long_term_instructions=["use simple explanations"],
    long_term_preferences=["healthy meals"],
    long_term_restrictions=["lactose intolerant"],
    long_term_situation=["beginner cook"]
)

print(json.dumps(delta3, indent=4))

{
    "new_long_term_instructions": [],
    "new_long_term_preferences": [],
    "new_long_term_restrictions": [],
    "new_long_term_situation": []
}


## Usage Scenario 5: Retrieving Data
We'll use embeddings and cosine similarity to retrieve relevant information.

### Input
```json
{
    "user_input": str,
    "long_term_instructions": List[str],
    "long_term_preferences": List[str],
    "long_term_restrictions": List[str],
    "long_term_situation": List[str],
}
```
### Output
```json
{
    "instructions": List[str],
    "preferences": List[str],
    "restrictions": List[str],
    "situation": List[str],
}
```

### Implementation

In [98]:
import numpy as np

CRITICAL_KEYWORDS = {
    "vegan", "vegetarian", "pescatarian", "gluten-free",
    "dairy-free", "lactose intolerant", "nut allergy", "peanut allergy"
}

def cosine_similarity(a: np.ndarray, b: np.ndarray) -> float:
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def update_profile_with_similarity(
    user_input: str,
    long_term_instructions: list[str],
    long_term_preferences: list[str],
    long_term_restrictions: list[str],
    long_term_situation: list[str],
    top_k: int = 5
):

    query_emb_resp = client.embeddings.create(
        model="text-embedding-3-small",
        input=user_input
    )
    query_embedding = np.array(query_emb_resp.data[0].embedding)


    def embed_items(items):
        if not items:
            return []
        resp = client.embeddings.create(
            model="text-embedding-3-small",
            input=items
        )
        return [np.array(d.embedding) for d in resp.data]


    instructions_emb = embed_items(long_term_instructions)
    preferences_emb = embed_items(long_term_preferences)
    restrictions_emb = embed_items(long_term_restrictions)
    situation_emb = embed_items(long_term_situation)

    def select_top(items, embeddings):
        if not items:
            return []
        sims = [cosine_similarity(query_embedding, emb) for emb in embeddings]
        top_indices = np.argsort(sims)[-top_k:][::-1]
        selected = [items[i] for i in top_indices]


        selected_set = set(selected)
        for item in items:
            for keyword in CRITICAL_KEYWORDS:
                if keyword.lower() in item.lower():
                    selected_set.add(item)
        return list(selected_set)

    return {
        "instructions": select_top(long_term_instructions, instructions_emb),
        "preferences": select_top(long_term_preferences, preferences_emb),
        "restrictions": select_top(long_term_restrictions, restrictions_emb),
        "situation": select_top(long_term_situation, situation_emb),
    }


### Testing

In [100]:
result1 = update_profile_with_similarity(
    user_input="I want a quick healthy dinner",
    long_term_instructions=["use simple explanations", "notify if food looks spoiled"],
    long_term_preferences=["healthy meals", "quick recipes", "gluten-free"],
    long_term_restrictions=["lactose intolerant", "peanut allergy"],
    long_term_situation=["beginner cook", "microwave only at work"],
    top_k=2
)
print(json.dumps(result1, indent=4))


{
    "instructions": [
        "notify if food looks spoiled",
        "use simple explanations"
    ],
    "preferences": [
        "quick recipes",
        "healthy meals",
        "gluten-free"
    ],
    "restrictions": [
        "lactose intolerant",
        "peanut allergy"
    ],
    "situation": [
        "microwave only at work",
        "beginner cook"
    ]
}


In [101]:
result2 = update_profile_with_similarity(
    user_input="I want a dessert that is also vegan and quick",
    long_term_instructions=["use simple explanations", "avoid using sugar heavily"],
    long_term_preferences=["sweet dishes", "spicy food", "vegan"],
    long_term_restrictions=["dairy-free", "nut allergy"],
    long_term_situation=["beginner cook", "stovetop only"],
    top_k=3
)
print(json.dumps(result2, indent=4))


{
    "instructions": [
        "use simple explanations",
        "avoid using sugar heavily"
    ],
    "preferences": [
        "sweet dishes",
        "spicy food",
        "vegan"
    ],
    "restrictions": [
        "dairy-free",
        "nut allergy"
    ],
    "situation": [
        "stovetop only",
        "beginner cook"
    ]
}


In [102]:
result3 = update_profile_with_similarity(
    user_input="Looking to plan meals for the week using leftovers and fresh veggies",
    long_term_instructions=[
        "notify if food looks spoiled", "avoid complex knife techniques",
        "use simple explanations", "plan meals in advance",
        "batch cook when possible", "keep track of expiry dates"
    ],
    long_term_preferences=[
        "healthy meals", "quick recipes", "vegan", "gluten-free",
        "low sodium", "spicy food", "international cuisine"
    ],
    long_term_restrictions=[
        "lactose intolerant", "no shellfish", "peanut allergy", "tree nut allergy"
    ],
    long_term_situation=[
        "beginner cook", "microwave only at work", "limited fridge space", "limited kitchen utensils"
    ],
    top_k=4
)
print(json.dumps(result3, indent=4))


{
    "instructions": [
        "notify if food looks spoiled",
        "plan meals in advance",
        "batch cook when possible",
        "keep track of expiry dates"
    ],
    "preferences": [
        "healthy meals",
        "international cuisine",
        "vegan",
        "gluten-free",
        "quick recipes"
    ],
    "restrictions": [
        "lactose intolerant",
        "no shellfish",
        "peanut allergy",
        "tree nut allergy"
    ],
    "situation": [
        "limited fridge space",
        "limited kitchen utensils",
        "microwave only at work",
        "beginner cook"
    ]
}


In [104]:
result5 = update_profile_with_similarity(
    user_input="I want to avoid allergens but still enjoy international cuisine",
    long_term_instructions=["notify if food looks spoiled", "use simple explanations"],
    long_term_preferences=["international cuisine", "gluten-free", "vegan", "quick recipes"],
    long_term_restrictions=["peanut allergy", "dairy-free"],
    long_term_situation=["beginner cook"],
    top_k=2
)
print(json.dumps(result5, indent=4))


{
    "instructions": [
        "notify if food looks spoiled",
        "use simple explanations"
    ],
    "preferences": [
        "gluten-free",
        "international cuisine",
        "vegan"
    ],
    "restrictions": [
        "dairy-free",
        "peanut allergy"
    ],
    "situation": [
        "beginner cook"
    ]
}


## Usage Scenario 6: Using Feedback to Alter Stored Items
Depending on a user's response to a recipe, we can generate long-term information.

### Input
```json
{
    "made_status": str,
    "rating": int, // out of ten
    "comments": str,
    "recipe": {
        "name": str
        "ingredients": List[str],
        "steps": List[str]
    },
    "long_term_instructions": List[str],
    "long_term_preferences": List[str],
    "long_term_restrictions": List[str],
    "long_term_situation": List[str],
}
```
### Output
```json
{
    "long_term_instructions": List[str],
    "long_term_preferences": List[str],
    "long_term_restrictions": List[str],
    "long_term_situation": List[str],
}
```

### Implementation

In [105]:
def update_long_term_from_feedback(
    made_status: str,
    rating: int,
    comments: str,
    recipe: dict,
    long_term_instructions: list[str],
    long_term_preferences: list[str],
    long_term_restrictions: list[str],
    long_term_situation: list[str]
):
    
    user_message = f"""
User Feedback:
- Made status: {made_status}
- Rating: {rating}/10
- Comments: {comments}

Recipe:
- Name: {recipe.get('name')}
- Ingredients: {', '.join(recipe.get('ingredients', []))}
- Steps: {', '.join(recipe.get('steps', []))}

Current long-term data:
- Instructions: {', '.join(long_term_instructions)}
- Preferences: {', '.join(long_term_preferences)}
- Restrictions: {', '.join(long_term_restrictions)}
- Situation: {', '.join(long_term_situation)}

Task:
Based on this feedback, update the long-term preferences, restrictions, and situation conservatively.
Only add new instructions; do not remove existing ones.
Return JSON with keys: long_term_instructions, long_term_preferences, long_term_restrictions, long_term_situation.
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_message}
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "long_term_update",
                "schema": {
                    "type": "object",
                    "properties": {
                        "long_term_instructions": {"type": "array", "items": {"type": "string"}},
                        "long_term_preferences": {"type": "array", "items": {"type": "string"}},
                        "long_term_restrictions": {"type": "array", "items": {"type": "string"}},
                        "long_term_situation": {"type": "array", "items": {"type": "string"}}
                    },
                    "required": ["long_term_instructions", "long_term_preferences", "long_term_restrictions", "long_term_situation"]
                }
            }
        }
    )

    return json.loads(response.choices[0].message.content)


### Testing

In [106]:
feedback1 = {
    "made_status": "made",
    "rating": 9,
    "comments": "I really liked this recipe, especially the quick prep and the vegan-friendly ingredients.",
    "recipe": {
        "name": "Vegan Stir-Fry",
        "ingredients": ["tofu", "broccoli", "soy sauce", "garlic", "ginger"],
        "steps": ["chop vegetables", "fry tofu", "add sauce and vegetables", "stir-fry 5 min"]
    },
    "long_term_instructions": ["use simple explanations"],
    "long_term_preferences": ["quick recipes", "healthy meals"],
    "long_term_restrictions": ["peanut allergy"],
    "long_term_situation": ["beginner cook"]
}

updated1 = update_long_term_from_feedback(**feedback1)
print(json.dumps(updated1, indent=4))


{
    "long_term_instructions": [
        "use simple explanations"
    ],
    "long_term_preferences": [
        "quick recipes",
        "healthy meals",
        "vegan-friendly ingredients"
    ],
    "long_term_restrictions": [
        "peanut allergy"
    ],
    "long_term_situation": [
        "beginner cook"
    ]
}


In [107]:
feedback2 = {
    "made_status": "made",
    "rating": 4,
    "comments": "The recipe was too spicy for me, but otherwise okay.",
    "recipe": {
        "name": "Spicy Chili",
        "ingredients": ["beans", "tomatoes", "chili peppers", "onion", "garlic"],
        "steps": ["soak beans", "cook vegetables", "add spices", "simmer 30 min"]
    },
    "long_term_instructions": ["use simple explanations"],
    "long_term_preferences": ["quick recipes"],
    "long_term_restrictions": ["gluten-free"],
    "long_term_situation": ["beginner cook"]
}

updated2 = update_long_term_from_feedback(**feedback2)
print(json.dumps(updated2, indent=4))


{
    "long_term_instructions": [
        "provide spice level adjustments in recipes"
    ],
    "long_term_preferences": [],
    "long_term_restrictions": [],
    "long_term_situation": [
        "prefers milder flavors"
    ]
}


In [108]:
feedback3 = {
    "made_status": "tried",
    "rating": 6,
    "comments": "It was okay, but the prep took too long.",
    "recipe": {
        "name": "Homemade Pasta",
        "ingredients": ["flour", "eggs", "olive oil", "salt"],
        "steps": ["mix dough", "knead", "roll", "cut pasta", "boil 5 min"]
    },
    "long_term_instructions": ["notify if food looks spoiled"],
    "long_term_preferences": ["healthy meals"],
    "long_term_restrictions": [],
    "long_term_situation": ["beginner cook"]
}

updated3 = update_long_term_from_feedback(**feedback3)
print(json.dumps(updated3, indent=4))


{
    "long_term_instructions": [
        "suggest time-saving techniques in recipes",
        "offer shortcuts for lengthy prep processes"
    ],
    "long_term_preferences": [
        "quick meals",
        "meals with simple steps"
    ],
    "long_term_restrictions": [],
    "long_term_situation": [
        "prefers time-efficient cooking methods",
        "still considered a beginner cook"
    ]
}


In [109]:
feedback5 = {
    "made_status": "made",
    "rating": 8,
    "comments": "Simple and easy to follow, very enjoyable.",
    "recipe": {
        "name": "Scrambled Eggs",
        "ingredients": ["eggs", "salt", "butter"],
        "steps": ["crack eggs", "beat", "cook on skillet 5 min"]
    },
    "long_term_instructions": [],
    "long_term_preferences": [],
    "long_term_restrictions": [],
    "long_term_situation": []
}

updated5 = update_long_term_from_feedback(**feedback5)
print(json.dumps(updated5, indent=4))


{
    "long_term_instructions": [
        "Ensure instructions are simple and easy to follow",
        "Include estimated cooking times"
    ],
    "long_term_preferences": [
        "Enjoys simple and easy-to-follow recipes"
    ],
    "long_term_restrictions": [],
    "long_term_situation": []
}
